In [1]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
data = pd.read_excel('/content/drive/My Drive/dataset/dataset.xlsx', engine='openpyxl')

data['Text'] = data['Text'].str.lower()

In [6]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [7]:
def remove_stopwords(data):
  data['review without stopwords'] = data['Text'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

In [9]:
data_without_stopwords.head()

,URL,Text,Sentiment,review without stopwords,clean_review
0,http://www.imdb.com/title/tt0210075/usercomments,girlfight follows a project dwelling new york ...,POS,girlfight follows project dwelling new york hi...,girlfight follows project dwelling new york hi...
1,http://www.imdb.com/title/tt0337640/usercomments,hollywood north is an euphemism from the movie...,POS,hollywood north euphemism movie industry went ...,hollywood north euphemism movie industry went ...
2,http://www.imdb.com/title/tt0303549/usercomments,that '70s show is definitely the funniest show...,POS,'70s show definitely funniest show currently t...,70s show definitely funniest show currently t...
3,http://www.imdb.com/title/tt0716825/usercomments,"9/10- 30 minutes of pure holiday terror. okay,...",POS,"9/10- 30 minutes pure holiday terror. okay, no...",9 10 30 minutes pure holiday terror okay no...
4,http://www.imdb.com/title/tt0182225/usercomments,"a series of random, seemingly insignificant th...",POS,"series random, seemingly insignificant thefts ...",series random seemingly insignificant thefts ...


In [13]:
reviews_list = []
for i in range(len(data_without_stopwords['clean_review'])):
  reviews_list.append(data_without_stopwords['clean_review'][i])
 
sentiment = data_without_stopwords['Sentiment']

In [16]:
y = np.array(list(map(lambda x: 1 if x=="POS" else 0, sentiment)))

X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

In [17]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [21]:
type(words_to_index)

dict

In [18]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

Download and mount the file glove.6B.50d.txt
Links to download: (download from any one of these) 
https://www.kaggle.com/watts2/glove6b50dtxt, 
https://nlp.stanford.edu/projects/glove/

Upload the .zip file, and extract it here. :)

In [25]:
# !unzip '/content/drive/My Drive/dataset/archive.zip' -d '/content/drive/My Drive/dataset/'

Archive:  /content/drive/My Drive/dataset/archive.zip
  inflating: /content/drive/My Drive/dataset/glove.6B.50d.txt  


In [26]:
word_to_vec_map = read_glove_vector('/content/drive/My Drive/dataset/glove.6B.50d.txt')

maxLen = 150

In [30]:
word_to_vec_map['moon'].shape[0]

50

In [32]:
len(words_to_index)

23418

In [46]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index - 1, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [47]:
def movie_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [48]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [59]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [49]:
X_train_indices.shape

(1599, 150)

In [56]:
model = movie_rating(X_train_indices.shape[1])

In [57]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
25/25 [==============================] - 30s 970ms/step - loss: 0.6958 - accuracy: 0.4747
Epoch 2/15
25/25 [==============================] - 24s 974ms/step - loss: 0.6905 - accuracy: 0.5255
Epoch 3/15
25/25 [==============================] - 24s 966ms/step - loss: 0.6921 - accuracy: 0.5104
Epoch 4/15
25/25 [==============================] - 25s 990ms/step - loss: 0.6918 - accuracy: 0.4940
Epoch 5/15
25/25 [==============================] - 25s 987ms/step - loss: 0.6904 - accuracy: 0.5258
Epoch 6/15
25/25 [==============================] - 25s 985ms/step - loss: 0.6926 - accuracy: 0.5140
Epoch 7/15
25/25 [==============================] - 25s 993ms/step - loss: 0.6900 - accuracy: 0.5174
Epoch 8/15
25/25 [==============================] - 25s 993ms/step - loss: 0.6880 - accuracy: 0.5206
Epoch 9/15
25/25 [==============================] - 25s 1s/step - loss: 0.6907 - accuracy: 0.5075
Epoch 10/15
25/25 [==============================] - 24s 969ms/step - loss: 0.6911 - accuracy:

In [60]:
model.evaluate(X_test_indices, Y_test)

13/13 [==============================] - 3s 141ms/step - loss: 0.6923 - accuracy: 0.5275


[0.6922664642333984, 0.5274999737739563]

In [62]:
preds = model.predict(X_test_indices)

n = np.random.randint(0,len(X_test))

# X_test[n]

if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')

precicted sentiment : negative
correct sentiment : negative


In [65]:
Y_test.shape

(400,)

In [66]:
preds.shape

(400, 1)

In [69]:
Y_test[:5]

array([1, 1, 1, 1, 1])

In [70]:
preds[:5]

array([[0.5375193 ],
       [0.5298851 ],
       [0.48587412],
       [0.4425039 ],
       [0.53750813]], dtype=float32)

In [74]:
def get_pred(preds):
  if preds > 0.5:
    return 1
  else: 
    return 0

In [75]:
predictions = [get_pred(x) for x in preds]

In [77]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.49      0.35      0.41       187
           1       0.54      0.69      0.61       213

    accuracy                           0.53       400
   macro avg       0.52      0.52      0.51       400
weighted avg       0.52      0.53      0.51       400

